In [1]:
import camelot
data = camelot.read_pdf('timetable.pdf', pages='6-25')

In [2]:
import pandas
datadf = data[0].df
for i in range(1,20):
    datadf = datadf.append(data[i].df)
datadf

,0,1,2,3,4,5,6,7,8,9
0,COM \nCODE,CLASS \nNUMBER \nAS IN ERP,COURSENO,COURSETITLE,L P U,CREDIT \nSEC,STAT,INSTRUCTOR IN CHARGE/Instructor,DAYS/ H,COMPRE DATE
1,021112,,1011 BIO F211,BIOLOGICAL CHEMISTRY,3 0 3,L,1,RAVIPRASAD ADURI/ Meenal Kowshik/ Rajesh Mehrotra,T TH S 3,09/12/20 (FN)
2,021112,,1012 BIO F211,BIOLOGICAL CHEMISTRY,3 0 3,T,1,Raviprasad Aduri/ Meenal Kowshik/ Rajesh Mehrotra,T 10,
3,021114,,1241 BIO F212,MICROBIOLOGY,3 1 4,L,1,JUDITH MARIA BRAGANCA/ Utpal Roy/ Vijayashree ...,T TH S 4,03/12/20 (FN)
4,021114,,1242 BIO F212,MICROBIOLOGY,3 1 4,P,1,"Judith Maria Braganca, Utpal Roy, Vijayashree ...",T TH 9,
5,021113,,1013 BIO F213,CELL BIOLOGY,3 0 3,L,1,ANGSHUMAN SARKAR/ Indrani Talukdar,M W F 3,05/12/20 (AN)
6,021113,,1014 BIO F213,CELL BIOLOGY,3 0 3,T,1,Angshuman Sarkar/ Indrani Talukdar,T 8,
7,021115,,1015 BIO F214,INTEGRATED BIOLOGY,3 0 3,L,1,SUMIT BISWAS/ Arnab Banerjee/ Rajesh Mehrotra,M W F 2,07/12/20 (FN)
8,021115,,1016 BIO F214,INTEGRATED BIOLOGY,3 0 3,T,1,Sumit Biswas/ Arnab Banerjee/ Rajesh Mehrotra,F 10,
9,021651,,1154 BIO F266,STUDY PROJECT,3,R,1,SRIKANTH MUTNURI,,


In [3]:
df = datadf.copy(deep=True)
df = df.drop(index=[0]).drop(columns=[0,1]).reset_index(drop=True)
df = df.rename(columns={2:'course_code', 3:'course_name', 4:'units', 5:'ltp', 6:'sec', 7:'instructor', 8:'time', 9:'compre'})
df['course_code'] = df['course_code'].apply(lambda x : x.split()[1]+'_'+x.split()[2])

In [4]:
df = df.dropna(subset=['time'])
df.iloc[pandas.Index(df['time']).get_loc('T TH 5-6:30 PM')]['time'] = 'T TH 10 11'
df.iloc[pandas.Index(df['time']).get_loc('TH S 12-1:30 PM')]['time'] = 'TH S 5 6'

In [5]:
days = {'M':-1, 'T':11, 'W':23, 'TH':35, 'F':47, 'S':59}

def reduce(code):
    codes = [code]
    for i in range(len(code)-1):
        if code[i] not in days and code[i+1] in days:
            codes = [(code[0:i+1])] + reduce(code[i+1:])
    return codes

def decode(codes):
    time = [0] * 72
    for code in codes:
        hr = []
        dy = []
        for i in code:
            if i in days:
                dy.append(i)
            else:
                hr.append(i)             
        for d in dy:
            for h in hr:
                time[days[d] + int(h)] = 1
    return time

In [6]:
df['timebool'] = df['time'].apply(lambda code : decode(reduce(code.split())))
df

,course_code,course_name,units,ltp,sec,instructor,time,compre,timebool
0,BIO_F211,BIOLOGICAL CHEMISTRY,3 0 3,L,1,RAVIPRASAD ADURI/ Meenal Kowshik/ Rajesh Mehrotra,T TH S 3,09/12/20 (FN),"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
1,BIO_F211,BIOLOGICAL CHEMISTRY,3 0 3,T,1,Raviprasad Aduri/ Meenal Kowshik/ Rajesh Mehrotra,T 10,,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,BIO_F212,MICROBIOLOGY,3 1 4,L,1,JUDITH MARIA BRAGANCA/ Utpal Roy/ Vijayashree ...,T TH S 4,03/12/20 (FN),"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,BIO_F212,MICROBIOLOGY,3 1 4,P,1,"Judith Maria Braganca, Utpal Roy, Vijayashree ...",T TH 9,,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,BIO_F213,CELL BIOLOGY,3 0 3,L,1,ANGSHUMAN SARKAR/ Indrani Talukdar,M W F 3,05/12/20 (AN),"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5,BIO_F213,CELL BIOLOGY,3 0 3,T,1,Angshuman Sarkar/ Indrani Talukdar,T 8,,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6,BIO_F214,INTEGRATED BIOLOGY,3 0 3,L,1,SUMIT BISWAS/ Arnab Banerjee/ Rajesh Mehrotra,M W F 2,07/12/20 (FN),"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7,BIO_F214,INTEGRATED BIOLOGY,3 0 3,T,1,Sumit Biswas/ Arnab Banerjee/ Rajesh Mehrotra,F 10,,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
8,BIO_F266,STUDY PROJECT,3,R,1,SRIKANTH MUTNURI,,,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
9,BIO_F311,RECOMBINANT DNA TECH,3 0 3,L,1,SAMIT CHATTOPADHYAY,M W F 8,14/12/20 (FN),"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."


In [33]:
df['instructor'] = df['instructor'].apply(lambda name : name.replace('/',',').replace('\n','').upper())
df['sec'] = df['ltp']+df['sec']
df['time'] = df['time'].apply(lambda x : x.replace(' ',''))

In [98]:
df1 = df.copy(deep=True)

In [99]:
df1

,course_code,course_name,units,ltp,sec,instructor,time,compre,timebool
0,BIO_F211,BIOLOGICAL CHEMISTRY,3 0 3,L,L1,"RAVIPRASAD ADURI, MEENAL KOWSHIK, RAJESH MEHROTRA",TTHS3,09/12/20 (FN),"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
1,BIO_F211,BIOLOGICAL CHEMISTRY,3 0 3,T,T1,"RAVIPRASAD ADURI, MEENAL KOWSHIK, RAJESH MEHROTRA",T10,,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,BIO_F212,MICROBIOLOGY,3 1 4,L,L1,"JUDITH MARIA BRAGANCA, UTPAL ROY, VIJAYASHREE ...",TTHS4,03/12/20 (FN),"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,BIO_F212,MICROBIOLOGY,3 1 4,P,P1,"JUDITH MARIA BRAGANCA, UTPAL ROY, VIJAYASHREE ...",TTH9,,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,BIO_F213,CELL BIOLOGY,3 0 3,L,L1,"ANGSHUMAN SARKAR, INDRANI TALUKDAR",MWF3,05/12/20 (AN),"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5,BIO_F213,CELL BIOLOGY,3 0 3,T,T1,"ANGSHUMAN SARKAR, INDRANI TALUKDAR",T8,,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6,BIO_F214,INTEGRATED BIOLOGY,3 0 3,L,L1,"SUMIT BISWAS, ARNAB BANERJEE, RAJESH MEHROTRA",MWF2,07/12/20 (FN),"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7,BIO_F214,INTEGRATED BIOLOGY,3 0 3,T,T1,"SUMIT BISWAS, ARNAB BANERJEE, RAJESH MEHROTRA",F10,,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
8,BIO_F266,STUDY PROJECT,3,R,R1,SRIKANTH MUTNURI,,,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
9,BIO_F311,RECOMBINANT DNA TECH,3 0 3,L,L1,SAMIT CHATTOPADHYAY,MWF8,14/12/20 (FN),"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."


In [100]:
def check_sec(s1,s2):
    for i in range(72):
        if s1['timebool'][i] and s2['timebool'][i]:
            return False
    return True
    
def merge_sec(s1,s2):    
    ans = s1.copy(deep=True)
    ans['sec'] += '+' + s2['sec']
    ans['time'] += '+' + s2['time']
    ans['instructor'] += '+' + s2['instructor']
    ans['timebool'] = [ans['timebool'][i] or s2['timebool'][i] for i in range(72)]
    return ans

def merge_types(df1,df2):
    merged = pandas.DataFrame(columns=df1.columns)
    for i in range(len(df1)):
        for j in range(len(df2)):
            if check_sec(df1.iloc[i],df2.iloc[j]):
                merged = merged.append(merge_sec(df1.iloc[i],df2.iloc[j]))
    return merged

def merge_crs(crs):    
    types={}    
    for name,group in crs.groupby(['ltp']):
        types[name] = group
    
    keysinorder = ['I','R','L','T','P']
    merged = pandas.DataFrame(columns=crs.columns)
    for k in keysinorder:
        if k in types.keys():
            if merged.empty:
                merged = types[k].copy(deep=True)
            else:
                merged = merge_types(merged,types[k])
                    
    return merged

In [101]:
df2 = df1.groupby(['course_code']).apply(merge_crs)
df2

,,course_code,course_name,units,ltp,sec,instructor,time,compre,timebool
course_code,,,,,,,,,,
BIOT_F346,34,BIOT_F346,GENOMICS,3 0 3,L,L1,"TUSAR TIRTHA SAHA, SANDHYA MEHROTRA",MWF7,12/12/20 (AN),"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
BIO_F211,0,BIO_F211,BIOLOGICAL CHEMISTRY,3 0 3,L,L1+T1,"RAVIPRASAD ADURI, MEENAL KOWSHIK, RAJESH MEHRO...",TTHS3+T10,09/12/20 (FN),"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
BIO_F212,2,BIO_F212,MICROBIOLOGY,3 1 4,L,L1+P1,"JUDITH MARIA BRAGANCA, UTPAL ROY, VIJAYASHREE ...",TTHS4+TTH9,03/12/20 (FN),"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
BIO_F213,4,BIO_F213,CELL BIOLOGY,3 0 3,L,L1+T1,"ANGSHUMAN SARKAR, INDRANI TALUKDAR+ANGSHUMAN S...",MWF3+T8,05/12/20 (AN),"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
BIO_F214,6,BIO_F214,INTEGRATED BIOLOGY,3 0 3,L,L1+T1,"SUMIT BISWAS, ARNAB BANERJEE, RAJESH MEHROTRA+...",MWF2+F10,07/12/20 (FN),"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
BIO_F266,8,BIO_F266,STUDY PROJECT,3,R,R1,SRIKANTH MUTNURI,,,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
BIO_F311,9,BIO_F311,RECOMBINANT DNA TECH,3 0 3,L,L1+T1,SAMIT CHATTOPADHYAY+SAMIT CHATTOPADHYAY,MWF8+F10,14/12/20 (FN),"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
BIO_F312,11,BIO_F312,PLANT PHYSIOLOGY,3 0 3,L,L1+T1,"SANDHYA MEHROTRA, KUNDAN KUMAR+SANDHYA MEHROTR...",TTHS2+T9,01/12/20 (AN),"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
BIO_F313,13,BIO_F313,ANIMAL PHYSIOLOGY,3 0 3,L,L1+T1,DIBAKAR CHAKRABARTY+DIBAKAR CHAKRABARTY,MWF9+TH10,11/12/20 (FN),"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."


In [106]:
df2 = df2.set_index(pandas.Index(range(len(df2))))

In [107]:
datadict = dict()
def make_dict(crs):
    
    secs = dict()
    for s in crs.index :
        sec = {
            'instructor' : crs['instructor'][s],
            'time' : crs['timebool'][s],
            'timecode' : crs['time'][s],
            'compre' : crs.iloc[0]['compre']
        }
        secs[crs['sec'][s]] = sec
        
    details = {
        'name' : crs.iloc[0]['course_name'],
        'units' : crs.iloc[0]['units'],
        'secnos' : len(secs),
        'sec' : secs
    }
    
    datadict[crs.iloc[0]['course_code']] = details

In [108]:
df2.groupby('course_code').apply(make_dict)

""


In [109]:
datadict

{'BIOT_F346': {'name': 'GENOMICS',
  'units': '3 0 3',
  'secnos': 1,
  'sec': {'L1': {'instructor': 'TUSAR TIRTHA SAHA, SANDHYA MEHROTRA',
    'time': [0,
     0,
     0,
     0,
     0,
     0,
     1,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     1,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     1,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0],
    'timecode': 'MWF7',
    'compre': '12/12/20 (AN)'}}},
 'BIO_F211': {'name': 'BIOLOGICAL CHEMISTRY',
  'units': '3 0 3',
  'secnos': 1,
  'sec': {'L1+T1': {'instructor': 'RAVIPRASAD ADURI, MEENAL KOWSHIK, RAJESH MEHROTRA+RAVIPRASAD ADURI, MEENAL KOWSHIK, RAJESH MEHROTRA',

In [110]:
import json
with open("datadictnew.json", 'w') as f :
    json.dump(datadict, f)

In [111]:
datadictmin = dict(datadict)
for crs in datadictmin.values() :
    del crs['units']
    for s in crs['sec'].values() :
        del s['time'],s['compre']
datadictmin

{'BIOT_F346': {'name': 'GENOMICS',
  'secnos': 1,
  'sec': {'L1': {'instructor': 'TUSAR TIRTHA SAHA, SANDHYA MEHROTRA',
    'timecode': 'MWF7'}}},
 'BIO_F211': {'name': 'BIOLOGICAL CHEMISTRY',
  'secnos': 1,
  'sec': {'L1+T1': {'instructor': 'RAVIPRASAD ADURI, MEENAL KOWSHIK, RAJESH MEHROTRA+RAVIPRASAD ADURI, MEENAL KOWSHIK, RAJESH MEHROTRA',
    'timecode': 'TTHS3+T10'}}},
 'BIO_F212': {'name': 'MICROBIOLOGY',
  'secnos': 1,
  'sec': {'L1+P1': {'instructor': 'JUDITH MARIA BRAGANCA, UTPAL ROY, VIJAYASHREE NAYAK+JUDITH MARIA BRAGANCA, UTPAL ROY, VIJAYASHREE NAYAK, SWETHA RAMESH, DEVIKA N NAGAR, ZANTYE PRANJITA GANESH, ZAKANE RESHMA NANASAHEB, SNEHA LATA, KODARKAR MAMTA MANGESH',
    'timecode': 'TTHS4+TTH9'}}},
 'BIO_F213': {'name': 'CELL BIOLOGY',
  'secnos': 1,
  'sec': {'L1+T1': {'instructor': 'ANGSHUMAN SARKAR, INDRANI TALUKDAR+ANGSHUMAN SARKAR, INDRANI TALUKDAR',
    'timecode': 'MWF3+T8'}}},
 'BIO_F214': {'name': 'INTEGRATED BIOLOGY',
  'secnos': 1,
  'sec': {'L1+T1': {'instructor

In [112]:
import json
with open("datadictminnew.json", 'w') as f :
    json.dump(datadictmin, f)